In [45]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random

In [46]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [47]:
response = requests.get(url)
response.status_code

200

In [48]:
soup = BeautifulSoup(response.content, "html.parser")

In [49]:
title=[]
artist=[]


title_list=soup.select("p.title-artist cite.title")
artist_list=soup.select("p.title-artist em.artist")

num_iter = len(soup.select("p.title-artist"))

for i in range(num_iter):
    title.append(title_list[i].get_text())
    artist.append(artist_list[i].get_text())

In [50]:
hot_songs = pd.DataFrame({"title":title,
                       "artist":artist})

In [51]:
hot_songs.head()

,title,artist
0,About Damn Time,Lizzo
1,Running Up That Hill (A Deal with God),Kate Bush
2,Irrelevant,P!nk
3,Y'all Life,Walker Hayes
4,Names,Tom MacDonald


In [61]:
hot_songs['title'] = hot_songs['title'].str.lower()
hot_songs['artist'] = hot_songs['artist'].str.lower()

In [70]:
# the function is taken from: https://albertoviciana.medium.com/my-first-song-recommender-96994af6d6bc


def hot():
    random_song=random.choice(hot_songs['title'])
    user_input = input ('Choose a song you like:')
    
    if user_input.lower() not in list(hot_songs['title']):
        print("Sorry, I have no recommendation")
    else:
        print("If you like the song " + "'" + user_input + "'" + " then listen to this next: " + "'" + random_song + "'")        

In [71]:
hot()

Choose a song you like:about damn time
If you like the song 'about damn time' then listen to this next: 'when i was your man'


In [72]:
hot()

Choose a song you like:sghyh
Sorry, I have no recommendation
